In [2]:
import pandas as pd
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [3]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,largeur_du_dos,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,...,hauteur_des_hanches,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille
3596,159.3088,22.7,58.998711,1.0,24.8920,56.7944,37.617400,87.7062,NaN,115.189000,...,80.7974,43.700700,47.9044,37.617400,129.89560,43.4086,77.5970,53.4924,73.9902,93.9038
20,171.9072,32.0,114.078388,1.0,28.7020,60.5028,42.113200,118.2116,NaN,121.589800,...,84.2010,46.101000,48.0060,42.113200,142.20190,54.4068,81.8896,74.0918,102.6922,94.2086
103,181.9910,36.0,93.199548,1.0,27.4066,59.9948,41.485936,105.1052,NaN,128.796549,...,86.4108,49.597886,49.8094,41.485936,145.86253,48.9966,84.5058,64.5922,100.3046,104.9020
2151,170.9928,64.0,61.008124,1.0,26.1874,61.3918,42.418000,93.5990,NaN,122.809000,...,86.8934,48.107600,42.0878,42.418000,138.69670,45.1104,82.9056,49.7078,82.1944,102.8954
3260,175.1076,40.0,71.667536,1.0,27.9908,63.5000,43.434000,91.6940,NaN,124.409200,...,86.5886,48.501300,45.8978,43.434000,138.69670,46.3042,84.5058,55.0926,77.8002,105.3084


In [4]:
caesar_male["largeur_des_hanches_ratio"] = caesar_male["largeur_des_hanches"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [5]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse']
    largeur_hanches = row['largeur_des_hanches_ratio']
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if largeur_hanches < 0.257434:
        cuisses = "fines"
    elif  0.257434 <=largeur_hanches<0.283177 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [6]:
# 1. Wrapper
class MultiTPOTWrapper:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        import numpy as np
        predictions = [m.predict(X) for m in self.models]
        return np.vstack(predictions).T

In [7]:
# 2. Pré-traitement
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_male[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [8]:
# 3. Chargement modèles
import joblib
import os

dossier_pipelines = "pipelines_complets"
models = []
for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        models.append(joblib.load(path))

In [19]:
import joblib
import os

dossier_pipelines = "pipelines_complets"
models = []

for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        model = joblib.load(path)
        models.append(model)
        print(f"✅ Modèle chargé : {filename}")

✅ Modèle chargé : pipeline_tpot_hauteur_d_entrejambe.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_la_taille.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_epaules.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_genoux.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_hanches.pkl
✅ Modèle chargé : pipeline_tpot_largeur_d_epaule.pkl
✅ Modèle chargé : pipeline_tpot_largeur_d_epaule_ratio.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_hanches.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_hanches_ratio.pkl
✅ Modèle chargé : pipeline_tpot_largeur_du_dos.pkl
✅ Modèle chargé : pipeline_tpot_longueur_avant.pkl
✅ Modèle chargé : pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✅ Modèle chargé : pipeline_tpot_longueur_du_bras.pkl
✅ Modèle chargé : pipeline_tpot_sex.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cheville.pkl
✅ Modèle chargé : pipelin

In [20]:
# 4. Pipeline complet
full_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("multi_model", MultiTPOTWrapper(models))
])

In [21]:
# 5. Sauvegarde
joblib.dump(full_pipeline, "full_pipeline.pkl")

# 6. Prédiction
pipeline_loaded = joblib.load("full_pipeline.pkl")

In [22]:
# Suppose que tu as déjà ton pipeline chargé
# pipeline_loaded est de type Pipeline(steps=[("preprocess", ...), ("multi_model", ...)])

# Accès au préprocesseur uniquement
preprocessor = pipeline_loaded.named_steps["preprocess"]

# Applique le transformateur au new_data brut
X = preprocessor.transform(new_data)

# Si tu veux voir sous forme de DataFrame
import pandas as pd

# Récupère les noms des colonnes après transformation
# Cela fonctionne uniquement si le ColumnTransformer est simple
try:
    feature_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X, columns=feature_names)
except:
    # fallback si les noms ne sont pas récupérables
    X_df = pd.DataFrame(X)

# Affiche les premières lignes
X_df.head()
X_df.columns

Index(['num__taille', 'num__age', 'num__weight', 'cat__categorie_ventre_moyen',
       'cat__categorie_ventre_plat', 'cat__categorie_ventre_rond',
       'cat__categorie_torse_fin', 'cat__categorie_torse_large',
       'cat__categorie_torse_moyen', 'cat__categorie_cuisses_fines',
       'cat__categorie_cuisses_larges', 'cat__categorie_cuisses_moyennes'],
      dtype='object')

In [28]:
# ✅ Créer un wrapper à partir des pipelines chargés
wrapper = MultiTPOTWrapper(models)
new_data = pd.DataFrame([{
    "taille": 187,
    "age": 33,
    "weight": 80,
    "categorie_ventre": "moyen",
    "categorie_torse": "large",
    "categorie_cuisses": "medium"
    
}])
# ✅ Prédictions
preds = wrapper.predict(new_data)


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\

In [29]:
# ✅ Extraire les noms des cibles
noms_variables = [f.replace("pipeline_tpot_", "").replace(".pkl", "") 
                  for f in sorted(os.listdir(dossier_pipelines)) if f.endswith(".pkl")]

# ✅ Associer les valeurs
predictions_dict = dict(zip(noms_variables, preds[0]))

# ✅ Affichage
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


📊 Prédictions :
 - hauteur_d_entrejambe : 0.83
 - hauteur_de_la_taille : 0.91
 - hauteur_de_poitrine : 0.83
 - hauteur_des_epaules : 0.71
 - hauteur_des_genoux : 0.69
 - hauteur_des_hanches : 0.78
 - largeur_d_epaule : 0.64
 - largeur_d_epaule_ratio : -0.05
 - largeur_des_epaules_a_l_horizontales : 0.34
 - largeur_des_hanches : 0.34
 - largeur_des_hanches_ratio : -0.18
 - largeur_du_dos : 0.64
 - longueur_avant : 0.69
 - longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 0.81
 - longueur_du_bras : 67.25
 - sex : 0.00
 - tour_de_cheville : 0.22
 - tour_de_cuisse : -0.16
 - tour_de_poitrine : -0.15
 - tour_de_taille : -0.19
 - tour_de_taille_ratio : -0.95
 - tour_du_cou : -0.11


In [36]:
import os
import joblib
import numpy as np
from sklearn.preprocessing import RobustScaler

# 📁 Dossier scalers
scalers_folder = "scalers"
os.makedirs(scalers_folder, exist_ok=True)

# Exemple : predictions_dict = {'hauteur_d_entrejambe': 0.83, 'hauteur_de_la_taille': 0.91, ...}

# Enregistrer un scaler pour chaque variable
for var, val in predictions_dict.items():
    scaler = RobustScaler()
    # On fit le scaler sur la valeur (simulateur)
    X = np.array([[val]])
    try:
        scaler.fit(X)
        scaler_path = os.path.join(scalers_folder, f"scaler_{var}.pkl")
        joblib.dump(scaler, scaler_path)
        print(f"✅ Scaler sauvegardé pour {var}")
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde du scaler pour {var} : {e}")

# Affichage des prédictions
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


✅ Scaler sauvegardé pour hauteur_d_entrejambe
✅ Scaler sauvegardé pour hauteur_de_la_taille
✅ Scaler sauvegardé pour hauteur_de_poitrine
✅ Scaler sauvegardé pour hauteur_des_epaules
✅ Scaler sauvegardé pour hauteur_des_genoux
✅ Scaler sauvegardé pour hauteur_des_hanches
✅ Scaler sauvegardé pour largeur_d_epaule
✅ Scaler sauvegardé pour largeur_d_epaule_ratio
✅ Scaler sauvegardé pour largeur_des_epaules_a_l_horizontales
✅ Scaler sauvegardé pour largeur_des_hanches
✅ Scaler sauvegardé pour largeur_des_hanches_ratio
✅ Scaler sauvegardé pour largeur_du_dos
✅ Scaler sauvegardé pour longueur_avant
✅ Scaler sauvegardé pour longueur_de_la_colonne_vertebrale_jusqu_au_poignet
✅ Scaler sauvegardé pour longueur_du_bras
✅ Scaler sauvegardé pour sex
✅ Scaler sauvegardé pour tour_de_cheville
✅ Scaler sauvegardé pour tour_de_cuisse
✅ Scaler sauvegardé pour tour_de_poitrine
✅ Scaler sauvegardé pour tour_de_taille
✅ Scaler sauvegardé pour tour_de_taille_ratio
✅ Scaler sauvegardé pour tour_du_cou
📊 Prédi

In [43]:
import joblib
import os
import numpy as np

def descale_predictions_robust(predictions_dict, dossier_scalers="scalers_y"):
    predictions_descalees = {}
    for nom, valeur in predictions_dict.items():
        if nom == "longueur_du_bras":
            # Cette variable est déjà dans les bonnes unités
            predictions_descalees[nom] = valeur
        else:
            path_scaler = os.path.join(dossier_scalers, f"scaler_{nom}.pkl")
            if os.path.exists(path_scaler):
                scaler = joblib.load(path_scaler)
                valeur_realiste = scaler.inverse_transform([[valeur]])[0, 0]
                predictions_descalees[nom] = round(valeur_realiste, 2)
            else:
                print(f"⚠️ Scaler manquant pour : {nom}")
                predictions_descalees[nom] = valeur  # au cas où on veut garder la valeur
    return predictions_descalees
    

# Déscalage
predictions_descalees = descale_predictions_robust(predictions_dict)

# Affichage des vraies valeurs
print("\n✅ Prédictions déscalées :")
for k, v in predictions_descalees.items():
    print(f"{k} : {v}")



✅ Prédictions déscalées :
hauteur_d_entrejambe : 87.12
hauteur_de_la_taille : 111.05
hauteur_de_poitrine : 135.9
hauteur_des_epaules : 152.59
hauteur_des_genoux : 52.28
hauteur_des_hanches : 93.9
largeur_d_epaule : 43.48
largeur_d_epaule_ratio : 0.23
largeur_des_epaules_a_l_horizontales : 49.44
largeur_des_hanches : 49.44
largeur_des_hanches_ratio : 0.26
largeur_du_dos : 43.48
longueur_avant : 27.78
longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 89.81
longueur_du_bras : 67.25038293388093
sex : 1.0
tour_de_cheville : 27.03
tour_de_cuisse : 57.57
tour_de_poitrine : 98.46
tour_de_taille : 85.81
tour_de_taille_ratio : 0.43
tour_du_cou : 46.77
